In [1]:
import mosek
print("MOSEK is successfully installed!")


MOSEK is successfully installed!


In [2]:
import cvxpy as cp

x = cp.Variable()
prob = cp.Problem(cp.Minimize(x**2), [x >= 1])
result = prob.solve(solver=cp.MOSEK)
print("Optimal value:", result)


Optimal value: 1.0000000137609035


In [23]:
import numpy as np

phi_uy = np.array([[9, 0],[0, 8]])

phi_ux = np.array([[2, 5, 0, 0],[0, 0, 9, 1]])
phi_xx = np.array([[1, 4, 0, 0],[2, 0, 0, 0], [0,0,9,6],[0,0,1,5]])
phi_xy = np.array([[2, 0],[1.5, 0],[0, 1],[0, 6]])

phi_xx_inv = np.linalg.inv(phi_xx)

K = phi_uy - phi_ux @ phi_xx_inv @ phi_xy
print(phi_uy)
print(K)

[[9 0]
 [0 8]]
[[ 5.9375      0.        ]
 [ 0.         13.79487179]]


In [24]:
zero_matrix = np.zeros_like(phi_xx)
phi_xx_extend = np.block([[phi_xx, zero_matrix],[phi_xx, phi_xx]])
zero_matrix = np.zeros_like(phi_uy)
phi_uy_extend = np.block([[phi_uy, zero_matrix],[phi_uy, phi_uy]])
zero_matrix = np.zeros_like(phi_ux)
phi_ux_extend = np.block([[phi_ux, zero_matrix],[phi_ux, phi_ux]])
zero_matrix = np.zeros_like(phi_xy)
phi_xy_extend = np.block([[phi_xy, zero_matrix],[phi_xy, phi_xy]])


phi_xx_extend_inv = np.linalg.inv(phi_xx_extend)

K_extend = phi_uy_extend - phi_ux_extend @ phi_xx_extend_inv @ phi_xy_extend
print(K_extend)

[[ 5.9375      0.          0.          0.        ]
 [ 0.         13.79487179  0.          0.        ]
 [ 5.9375      0.          5.9375      0.        ]
 [ 0.         13.79487179  0.         13.79487179]]


In [5]:
import numpy as np
import re

# 读取 txt 文件
file_path = "D:/sparks/Study/Research/2025-control/lowRankControl-main/OffdiagMatrices.txt"  # 请替换为你的实际文件路径

with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

# 解析矩阵
def extract_matrix(lines, start_marker, end_marker):
    """ 提取 txt 文件中给定标记的矩阵 """
    start_idx = None
    end_idx = None

    # 找到起始和结束索引
    for i, line in enumerate(lines):
        if start_marker in line:
            start_idx = i + 1
        elif end_marker in line and start_idx is not None:
            end_idx = i
            break

    if start_idx is None or end_idx is None:
        return None  # 如果找不到标记，返回 None

    # 提取矩阵数据
    matrix_data = []
    max_cols = 0

    for line in lines[start_idx:end_idx]:
        numbers = re.findall(r"[-+]?\d*\.\d+|[-+]?\d+", line)
        if numbers:
            row = [float(num) for num in numbers]
            matrix_data.append(row)
            max_cols = max(max_cols, len(row))  # 记录最大列数

    # 统一行的长度
    for i in range(len(matrix_data)):
        if len(matrix_data[i]) < max_cols:
            matrix_data[i] += [0.0] * (max_cols - len(matrix_data[i]))  # 用 0 补齐

    return np.array(matrix_data)

# 提取 F, L21, L12
F = extract_matrix(lines, "=== F matrix (closed-loop) ===", "===")
L21 = extract_matrix(lines, "=== L21 matrix (UAV2->UAV1) ===", "===")
L12 = extract_matrix(lines, "=== L12 matrix (UAV1->UAV2) ===", "===")

# 输出矩阵形状和示例
if F is not None:
    print("F matrix shape:", F.shape)
    print("F matrix sample:\n", F[:5, :5])

if L21 is not None:
    print("L21 matrix shape:", L21.shape)
    print("L21 matrix sample:\n", L21[:5, :5])

if L12 is not None:
    print("L12 matrix shape:", L12.shape)
    print("L12 matrix sample:\n", L12[:5, :5])


F matrix shape: (220, 9)
F matrix sample:
 [[-0.2105 -0.0358  0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.    ]]
L21 matrix shape: (66, 9)
L21 matrix sample:
 [[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [-0. -0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]


In [21]:
# %% [code]
import re
import numpy as np

def parse_matrix(matrix_str):
    """
    将形如：
        [[-0.2105 -0.0358  0.      0.   ... 0.    ]
         [-0.0048 -0.0354 -0.     -0.   ... 0.    ]
         ... ]
    的字符串转换为二维列表。
    
    说明：  
    - 矩阵中的每一行都被 [] 包围，但行内各数字之间没有逗号，仅以空白分隔。
    - 利用正则表达式先提取出所有被 [] 包围的行，再按空白拆分数字。
    """
    # 正则表达式：匹配每个由 [ 和 ] 包围的子串（不匹配内部的 [ 或 ]）
    row_strings = re.findall(r'\[([^\[\]]+)\]', matrix_str)
    matrix = []
    for row in row_strings:
        # 用 split() 按所有空白分割，过滤空字符串
        tokens = row.split()
        if tokens:  # 如果该行非空
            try:
                row_floats = [float(tok) for tok in tokens]
            except Exception as e:
                raise ValueError(f"转换行数据失败：{row}\n错误信息：{e}")
            matrix.append(row_floats)
    return matrix

def extract_matrix(text, name):
    """
    从完整文本中提取名称为 name 的矩阵，并返回 numpy 数组
    
    参数：
      text: 文件全部内容（字符串）
      name: 矩阵名称，例如 "F", "L21", "L12"
            对应标题分别为：
              "=== F matrix (closed-loop) ==="
              "=== L21 matrix (UAV2->UAV1) ==="
              "=== L12 matrix (UAV1->UAV2) ==="
    """
    # 构造正则表达式。  
    # 注意：这里使用非贪婪模式 .*? 以匹配标题后面的矩阵块。
    pattern = r"===\s*{} matrix.*?===\s*(\[\[.*?\]\])".format(name)
    match = re.search(pattern, text, re.DOTALL)
    if not match:
        raise ValueError(f"未找到矩阵 {name} 的匹配项")
    matrix_str = match.group(1)
    # 用自定义的 parse_matrix 解析字符串（不能直接用 ast.literal_eval）
    matrix_list = parse_matrix(matrix_str)
    return np.array(matrix_list)

# %% [code]
# 读取 txt 文件（请调整文件路径和编码，如有需要）
file_path = "D:/sparks/Study/Research/2025-control/lowRankControl-main/OffdiagMatrices.txt"  # 请替换为你的实际文件路径
with open(file_path, "r", encoding="utf-8") as f:
    content = f.read()

# 分别提取 F, L21, L12 三个矩阵
F   = extract_matrix(content, "F")
L21 = extract_matrix(content, "L21")
L12 = extract_matrix(content, "L12")

# 查看提取出的矩阵形状及部分数据
print("F 矩阵形状:", F.shape)
print("L21 矩阵形状:", L21.shape)
print("L12 矩阵形状:", L12.shape)

print("\nF 矩阵前两行：\n", F[:2])


F 矩阵形状: (44, 44)
L21 矩阵形状: (22, 22)
L12 矩阵形状: (22, 22)

F 矩阵前两行：
 [[-2.10480552e-01 -3.58100687e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-4.78769850e-03 -3.54280157e-02 -6.00000000e-10 -6.00000000e-10
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.0000

In [22]:
rank_F   = np.linalg.matrix_rank(F, tol=1e-7)
rank_L21 = np.linalg.matrix_rank(L21, tol=1e-7)
rank_L12 = np.linalg.matrix_rank(L12, tol=1e-7)

print("F 矩阵的秩:", rank_F)
print("L21 矩阵的秩:", rank_L21)
print("L12 矩阵的秩:", rank_L12)


F 矩阵的秩: 39
L21 矩阵的秩: 1
L12 矩阵的秩: 1


In [1]:
import numpy as np

# 定义一个简单的类，用来容纳属性 self.T 与矩阵 self.F
class Example:
    def __init__(self, T):
        self.T = T
        # 这里创建一个 8×8 的简单测试矩阵，元素 0~63
        # 当 T=1 时， 4*(T+1) = 8，恰好是个 8×8
        self.F = np.arange(64).reshape(8, 8)
    
    def extract_mutral_detection_communication_matrix(self, direction='21'):
        Tplus1 = self.T + 1
        block_row = 4
        block_col = 4
        
        # 最终 big_mat 的大小是 2*(T+1) 行 × 2*(T+1) 列
        big_mat = np.zeros((2*Tplus1, 2*Tplus1))

        for t in range(Tplus1):
            for tau in range(Tplus1):
                row_start = t * block_row
                col_start = tau * block_col

                # 取出 4×4 分块
                sub_4x4 = self.F[row_start : row_start + block_row,
                                  col_start : col_start + block_col]

                # direction == '21' -> L1 索引: 行 [0,1], 列 [1,3]
                # direction == '12' -> L2 索引: 行 [2,3], 列 [0,2]
                if direction == '21':
                    sub_2x2 = sub_4x4[np.ix_([0,1], [1,3])]
                else:  # direction == '12'
                    sub_2x2 = sub_4x4[np.ix_([2,3], [0,2])]

                # 拼装进 big_mat
                big_mat[t*2 : (t+1)*2, tau*2 : (tau+1)*2] = sub_2x2
        
        return big_mat

# 创建一个 T=1 的示例实例
ex = Example(T=1)

print("原始 8×8 矩阵 F：")
print(ex.F, "\n")

# direction='21' 的结果
out21 = ex.extract_mutral_detection_communication_matrix(direction='21')
print("结果 (direction='21'):")
print(out21, "\n")

# direction='12' 的结果
out12 = ex.extract_mutral_detection_communication_matrix(direction='12')
print("结果 (direction='12'):")
print(out12)


原始 8×8 矩阵 F：
[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55]
 [56 57 58 59 60 61 62 63]] 

结果 (direction='21'):
[[ 1.  3.  5.  7.]
 [ 9. 11. 13. 15.]
 [33. 35. 37. 39.]
 [41. 43. 45. 47.]] 

结果 (direction='12'):
[[16. 18. 20. 22.]
 [24. 26. 28. 30.]
 [48. 50. 52. 54.]
 [56. 58. 60. 62.]]
